In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
db_file = "../data/users.json"
output_file = "../data/contacts.csv"
max_codeplug = 200000

In [3]:
import json
import io
import pandas as pd
import csv

In [4]:
users = json.load(open(db_file))

In [5]:


users_buffer = io.StringIO(json.dumps(users.get("users")))

In [6]:
all_users = pd.read_json(users_buffer, orient="records")
all_users['country_index'] = all_users['country'].str.lower()
all_users['city_index'] = all_users['city'].str.lower()
all_users


,fname,name,country,callsign,city,surname,radio_id,id,remarks,state,country_index,city_index
0,Wayne,Wayne,Canada,VE3THW,Toronto,Edward,1023001,1023001,DMR,Ontario,canada,toronto
1,Mathieu,Mathieu,Canada,VA3ECM,Ottawa-Hull,Goulet,1023002,1023002,CCS7,Quebec,canada,ottawa-hull
2,Guy,Guy,Canada,VE3QC,Gloucester,Charron,1023003,1023003,CCS7,Ontario,canada,gloucester
3,Allan Timothy,Allan Timothy,Canada,VA3UZ,Sparta,Harvey,1023006,1023006,DMR,Ontario,canada,sparta
4,Hans Juergen,Hans Juergen,Canada,VA3BOC,Cornwall,Bockholt,1023007,1023007,,Ontario,canada,cornwall
...,...,...,...,...,...,...,...,...,...,...,...,...
209066,Daniel,Daniel,Uruguay,CX2RT,Maldonado-Lausana-,Mazzul,7486001,7486001,DMR,Maldonado,uruguay,maldonado-lausana-
209067,Luis,Luis,Uruguay,CX1RE,Maldonado,Olivera,7486002,7486002,DMR,Maldonado,uruguay,maldonado
209068,Mario Hector,Mario Hector,Uruguay,CX4RP,Piriapolis Maldonado,Paller Figueiras,7486003,7486003,DMR,Maldonado,uruguay,piriapolis maldonado
209069,E. Eugenio,E. Eugenio,Uruguay,CX7SS,Villa Serrana,De Marino,7489001,7489001,DMR,,uruguay,villa serrana


In [7]:
all_users = all_users.sort_values(by=["country_index", "radio_id"], ascending=[True, False])
all_users

,fname,name,country,callsign,city,surname,radio_id,id,remarks,state,country_index,city_index
60109,Leif,Leif,Aland Islands,OH0KCE,Finstrom,,2440005,2440005,None,,aland islands,finstrom
60106,Leif,Leif,Aland Islands,OH0KCE,Finstrom,,2440002,2440002,None,,aland islands,finstrom
199188,Djaballah,Djaballah,Algeria,7X2DD,Djelfa,Mohamed,6030009,6030009,DMR,,algeria,djelfa
199187,Amine,Amine,Algeria,7X2ET,Algiers KOUBA,Ait Chalal,6030008,6030008,DMR,,algeria,algiers kouba
199186,Rabeh,Rabeh,Algeria,7X2EZ,Algiers,Slimani,6030007,6030007,DMR,,algeria,algiers
...,...,...,...,...,...,...,...,...,...,...,...,...
199325,Jerry,Jerry,Zambia,9J2BS,Mufulira,,6450003,6450003,None,,zambia,mufulira
199324,Rob,Rob,Zambia,9J2RD,Ndola,,6450002,6450002,None,,zambia,ndola
199323,Gabi,Gabi,Zambia,9J2YO,Lusaka,,6450001,6450001,None,,zambia,lusaka
199394,Howard,Howard,Zimbabwe,Z21EK,Harare,,6480002,6480002,None,,zimbabwe,harare


In [8]:
callsign_per_country = all_users.groupby(by=["country_index"])["radio_id"].agg(["count"]).sort_values(by="count", ascending=False)
callsign_per_country

,count
country_index,
united states,93740
united kingdom,14197
germany,13393
italy,9534
spain,7124
...,...
macao,1
madagascar,1
mauretania,1


In [9]:
callsign_per_country["cumsum"] = callsign_per_country["count"].cumsum()
callsign_per_country

,count,cumsum
country_index,,
united states,93740,93740
united kingdom,14197,107937
germany,13393,121330
italy,9534,130864
spain,7124,137988
...,...,...
macao,1,209067
madagascar,1,209068
mauretania,1,209069


In [10]:
most_callsigns_per_country = callsign_per_country.query(f"cumsum <= {max_codeplug}")

most_callsigns_per_country = most_callsigns_per_country.reset_index()
most_callsigns_per_country

,country_index,count,cumsum
0,united states,93740,93740
1,united kingdom,14197,107937
2,germany,13393,121330
3,italy,9534,130864
4,spain,7124,137988
5,china,5845,143833
6,canada,5485,149318
7,brazil,3992,153310
8,australia,3559,156869
9,turkey,3413,160282


In [11]:
least_callsigns_per_country = callsign_per_country.query(f"cumsum > {max_codeplug}")
least_callsigns_per_country

,count,cumsum
country_index,,
croatia,510,200431
ukraine,508,200939
indonesia,490,201429
puerto rico,486,201915
india,463,202378
...,...,...
macao,1,209067
madagascar,1,209068
mauretania,1,209069


In [12]:

all_merged_users = all_users.merge(callsign_per_country, how="outer", on=["country_index"], indicator=True)

potential_codeplug_users = all_merged_users.query("_merge == 'both'").drop(columns=["_merge"])
extra_users = all_merged_users.query("_merge == 'left_only'").drop(columns=["_merge"])

In [13]:
codeplug_users = potential_codeplug_users.query(f"cumsum <= {max_codeplug}")
codeplug_users

,fname,name,country,callsign,city,surname,radio_id,id,remarks,state,country_index,city_index,count,cumsum
46,Levita,Levita,Argentina Republic,LU9FSD,Esperanza Santa Fe,Alejandro Angel,7229028,7229028,DMR,Santa Fe,argentina republic,esperanza santa fe,985,193761
47,Emilio Hugo,Emilio Hugo,Argentina Republic,LU2FEH,Pueblo Esther Sf 2126,Splendiani,7229027,7229027,DMR,Santa Fe,argentina republic,pueblo esther sf 2126,985,193761
48,Andres Vicente,Andres Vicente,Argentina Republic,LU4FOZ,Funes Sf 2132,Szyszko,7229026,7229026,DMR,Santa Fe,argentina republic,funes sf 2132,985,193761
49,Andres Vicente,Andres Vicente,Argentina Republic,LU4FOZ,Funes Sf 2132,Szyszko,7229025,7229025,DMR,Santa Fe,argentina republic,funes sf 2132,985,193761
50,Santiago Ivan,Santiago Ivan,Argentina Republic,LU1FT,2000 Rosario - Santa Fe,Zeano,7229024,7229024,DMR,Santa Fe,argentina republic,2000 rosario - santa fe,985,193761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208365,Brian,Brian,United States,KI6NRR,Santa Clarita,Kunkel,1106006,1106006,DMR,California,united states,santa clarita,93740,93740
208366,Brian,Brian,United States,KI6NRR,Santa Clarita,Kunkel,1106005,1106005,DMR,California,united states,santa clarita,93740,93740
208367,Raul G,Raul G,United States,N7BAR,Moreno Valley,Gonzalez,1106004,1106004,DMR,California,united states,moreno valley,93740,93740
208368,Frederic K,Frederic K,United States,KH7Y,Pine Grove,Honnold,1106003,1106003,DMR,California,united states,pine grove,93740,93740


In [14]:
delta_max = max_codeplug - len(codeplug_users)
delta_max

79

In [15]:
extra_cp_users = potential_codeplug_users.query(f"cumsum > {max_codeplug}").sort_values(by=["count", "radio_id"], ascending=[False, False])
extra_cp_users = extra_cp_users.head(delta_max)

In [16]:
codeplug_users = pd.concat([codeplug_users, extra_cp_users], ignore_index=True)
codeplug_users

,fname,name,country,callsign,city,surname,radio_id,id,remarks,state,country_index,city_index,count,cumsum
0,Levita,Levita,Argentina Republic,LU9FSD,Esperanza Santa Fe,Alejandro Angel,7229028,7229028,DMR,Santa Fe,argentina republic,esperanza santa fe,985,193761
1,Emilio Hugo,Emilio Hugo,Argentina Republic,LU2FEH,Pueblo Esther Sf 2126,Splendiani,7229027,7229027,DMR,Santa Fe,argentina republic,pueblo esther sf 2126,985,193761
2,Andres Vicente,Andres Vicente,Argentina Republic,LU4FOZ,Funes Sf 2132,Szyszko,7229026,7229026,DMR,Santa Fe,argentina republic,funes sf 2132,985,193761
3,Andres Vicente,Andres Vicente,Argentina Republic,LU4FOZ,Funes Sf 2132,Szyszko,7229025,7229025,DMR,Santa Fe,argentina republic,funes sf 2132,985,193761
4,Santiago Ivan,Santiago Ivan,Argentina Republic,LU1FT,2000 Rosario - Santa Fe,Zeano,7229024,7229024,DMR,Santa Fe,argentina republic,2000 rosario - santa fe,985,193761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,Damir,Damir,Croatia,9A5BDE,Split,Eterovic,2190437,2190437,DMR,,croatia,split,510,200431
199996,Viktor,Viktor,Croatia,9A3VU,Slavonski Brod,Jelkovic,2190436,2190436,DMR,,croatia,slavonski brod,510,200431
199997,Radio Club,Radio Club,Croatia,9A1HDE,Varazdin,Varazdin,2190435,2190435,DMR,,croatia,varazdin,510,200431
199998,Horvat,Horvat,Croatia,9A3TR,VARAZDIN,Zvonko,2190434,2190434,DMR,Varaždin,croatia,varazdin,510,200431


In [17]:
codeplug_users = codeplug_users.drop(columns=["country_index", "city_index", "count", "cumsum", "id"])


def join_name(u: pd.Series) -> str:
    if str(u.fname).endswith(u.surname) or u.surname is None or u.surname.strip().lower() in ["", "none"]:
        return u.fname
    else:
        return f"{u.fname} {u.surname}"

codeplug_users['full name'] = codeplug_users.apply(join_name, axis='columns')
codeplug_users

,fname,name,country,callsign,city,surname,radio_id,remarks,state,full name
0,Levita,Levita,Argentina Republic,LU9FSD,Esperanza Santa Fe,Alejandro Angel,7229028,DMR,Santa Fe,Levita Alejandro Angel
1,Emilio Hugo,Emilio Hugo,Argentina Republic,LU2FEH,Pueblo Esther Sf 2126,Splendiani,7229027,DMR,Santa Fe,Emilio Hugo Splendiani
2,Andres Vicente,Andres Vicente,Argentina Republic,LU4FOZ,Funes Sf 2132,Szyszko,7229026,DMR,Santa Fe,Andres Vicente Szyszko
3,Andres Vicente,Andres Vicente,Argentina Republic,LU4FOZ,Funes Sf 2132,Szyszko,7229025,DMR,Santa Fe,Andres Vicente Szyszko
4,Santiago Ivan,Santiago Ivan,Argentina Republic,LU1FT,2000 Rosario - Santa Fe,Zeano,7229024,DMR,Santa Fe,Santiago Ivan Zeano
...,...,...,...,...,...,...,...,...,...,...
199995,Damir,Damir,Croatia,9A5BDE,Split,Eterovic,2190437,DMR,,Damir Eterovic
199996,Viktor,Viktor,Croatia,9A3VU,Slavonski Brod,Jelkovic,2190436,DMR,,Viktor Jelkovic
199997,Radio Club,Radio Club,Croatia,9A1HDE,Varazdin,Varazdin,2190435,DMR,,Radio Club Varazdin
199998,Horvat,Horvat,Croatia,9A3TR,VARAZDIN,Zvonko,2190434,DMR,Varaždin,Horvat Zvonko


In [18]:
codeplug_users["radio_id_idx"] = codeplug_users["radio_id"].astype(int)
codeplug_users = codeplug_users.sort_values(by="radio_id_idx").drop(columns=["radio_id_idx"])

fullname = False
if fullname:
    codeplug_users = codeplug_users[["radio_id", "callsign", "Name", "city", "state", "country", "remarks"]]

    colnames = {
        'radio_id': "Radio ID",
        'callsign': "Callsign",
        'city': "City",
        'state': "State",
        'country': "Country",
        'remarks': "Remarks"
    }
else:
    codeplug_users =  codeplug_users[["radio_id", "callsign", "fname", "surname", "city", "state", "country"]]

    colnames = {
        'radio_id': "Radio ID",
        'callsign': "Callsign",
        'fname': "Name",
        'surname': "City",
        'city': "State",
        'state': "Country",
        'country': "Remarks"
    }

codeplug_users = codeplug_users.rename(columns=colnames)
codeplug_users

,Radio ID,Callsign,Name,City,State,Country,Remarks
18000,1023001,VE3THW,Wayne,Edward,Toronto,Ontario,Canada
17999,1023002,VA3ECM,Mathieu,Goulet,Ottawa-Hull,Quebec,Canada
17998,1023003,VE3QC,Guy,Charron,Gloucester,Ontario,Canada
17997,1023006,VA3UZ,Allan Timothy,Harvey,Sparta,Ontario,Canada
17996,1023007,VA3BOC,Hans Juergen,Bockholt,Cornwall,Ontario,Canada
...,...,...,...,...,...,...,...
18005,7308005,CA8PVT,Pedro,Vera Paredes,Punta Arenas,Magallanes,Chile
18004,7308006,CD8DQF,David,Aguilera,Punta Arenas,Magallanes,Chile
18003,7308007,CA8AIJ,Guido,Iceta,Porvenir,Magallanes,Chile
18002,7308008,CA8EVZ,Eduardo Hernan,Valenzuela Pavez,Punta Arenas,Magallanes,Chile


In [19]:
codeplug_users['No.'] = (codeplug_users.reset_index(drop=True).index + 1).to_list()
codeplug_users

,Radio ID,Callsign,Name,City,State,Country,Remarks,No.
18000,1023001,VE3THW,Wayne,Edward,Toronto,Ontario,Canada,1
17999,1023002,VA3ECM,Mathieu,Goulet,Ottawa-Hull,Quebec,Canada,2
17998,1023003,VE3QC,Guy,Charron,Gloucester,Ontario,Canada,3
17997,1023006,VA3UZ,Allan Timothy,Harvey,Sparta,Ontario,Canada,4
17996,1023007,VA3BOC,Hans Juergen,Bockholt,Cornwall,Ontario,Canada,5
...,...,...,...,...,...,...,...,...
18005,7308005,CA8PVT,Pedro,Vera Paredes,Punta Arenas,Magallanes,Chile,199996
18004,7308006,CD8DQF,David,Aguilera,Punta Arenas,Magallanes,Chile,199997
18003,7308007,CA8AIJ,Guido,Iceta,Porvenir,Magallanes,Chile,199998
18002,7308008,CA8EVZ,Eduardo Hernan,Valenzuela Pavez,Punta Arenas,Magallanes,Chile,199999


In [20]:
column_order = ["No.","Radio ID","Callsign","Name","City","State","Country","Remarks", "Call Type", "Call Alert"]
codeplug_users.loc[:, "Call Type"] = "Private Call"
codeplug_users.loc[:, "Call Alert"] = "None"

codeplug_users[column_order].to_csv(output_file,
                                    index=None, quoting=csv.QUOTE_ALL, line_terminator='\r\n')